<a href="https://colab.research.google.com/github/lrsilsbee/Machine-Learning/blob/main/Lauren_Silsbee_data_exploration_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
import warnings

warnings.filterwarnings('ignore')
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'studentsperformance'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

performance_df = pd.read_sql_query('select * from studentsperformance',con=engine)
engine.dispose()

Question 1: Are there any differences between the genders, ethnicities, and parental level of education with respect to their performances in exams?

In [ ]:
#Gender 
performance_df.groupby("gender").mean()[["math score", "reading score", "writing score"]]
#There certainly seem to be differences. Let's see if it is significant
from scipy.stats.stats import ttest_ind
ttest_ind(performance_df[performance_df.gender == "female"][["math score", "reading score", "writing score"]],
          performance_df[performance_df.gender == "male"][["math score", "reading score", "writing score"]])
print('Female and male students differ in their math, reading, and writing scores.')

#Ethnicity


performance_df.groupby("race/ethnicity").mean()[["math score", "reading score", "writing score"]]
#There is variation, see if it is significant:

ethnicities = performance_df["race/ethnicity"].unique()
for var in ["math score", "reading score", "writing score"]:
    print("Comparisons for  {}".format(var))
    for i in range(0, len(ethnicities)):
        for j in range(i+1, len(ethnicities)):
            print(
                "t-test between {0} and {1}:".format(ethnicities[i], ethnicities[j]))
            print(ttest_ind(
                performance_df[performance_df["race/ethnicity"]
                           == ethnicities[i]][var],
                performance_df[performance_df["race/ethnicity"] == ethnicities[j]][var]
            ))

print('******************************************')
#Parent education
# Test whether group differences are significant.
parent_ed = performance_df["parental level of education"].unique()
#grouped_df = student_df.groupby("parental level of education")
for var in ["math score", "reading score", "writing score"]:
    print("------------------------------------------------")
    print("Comparisons for variable: {}".format(var))
    print("------------------------------------------------")
    for i in range(0, len(parent_ed)):
        for j in range(i+1, len(parent_ed)):
            print(
                "t-test between groups {0} and {1}:".format(parent_ed[i], parent_ed[j]))
            print(ttest_ind(
                performance_df[performance_df["parental level of education"]
                           == parent_ed[i]][var],
                performance_df[performance_df["parental level of education"]
                           == parent_ed[j]][var]
            ))

Female and male students differ in their math, reading, and writing scores.
Comparisons for  math score
t-test between group B and group C:
Ttest_indResult(statistic=-0.7315669893534263, pvalue=0.4647708939167453)
t-test between group B and group A:
Ttest_indResult(statistic=0.9355076279747488, pvalue=0.3503415961659957)
t-test between group B and group D:
Ttest_indResult(statistic=-2.82845539712675, pvalue=0.004886197137104194)
t-test between group B and group E:
Ttest_indResult(statistic=-6.007397050552227, pvalue=5.007946047497971e-09)
t-test between group C and group A:
Ttest_indResult(statistic=1.5997220303217299, pvalue=0.11043810745588042)
t-test between group C and group D:
Ttest_indResult(statistic=-2.41858624746011, pvalue=0.015888349556016285)
t-test between group C and group E:
Ttest_indResult(statistic=-6.127642520822135, pvalue=1.927238849552764e-09)
t-test between group A and group D:
Ttest_indResult(statistic=-3.3466982429712298, pvalue=0.0009068636053543751)
t-test bet

Question 2: Are there any differences between the lunch types with respect to their performances in exams? If there are, how do you explain this?

In [ ]:
performance_df.groupby("lunch").mean()[["math score", "reading score", "writing score"]]
#looks like the kids on free/reduced lunch score lower
#lets see if it is significant 


print(ttest_ind(performance_df[performance_df.lunch == "free/reduced"][["math score", "reading score", "writing score"]],
          performance_df[performance_df.lunch == "standard"][["math score", "reading score", "writing score"]]))

print('Yes, this is a significant difference. It appears that the children from lower income homes score lower on the tests.')

Ttest_indResult(statistic=array([-11.83718047,  -7.45105647,  -8.0097842 ]), pvalue=array([2.41319560e-30, 2.00279665e-13, 3.18618958e-15]))
Yes, this is a significant difference. It appears that the children from lower income homes score lower on the tests.


Question 3: Does the test preparation course seem to have an effect on the exam performances?

In [ ]:
performance_df.head()
# 'test preparation course' 'none' or 'completed' 
performance_df.groupby("test preparation course").mean()[["math score", "reading score", "writing score"]]
print('Yes, there seems to be a difference.')

#Check for significance:
print(ttest_ind(performance_df[performance_df['test preparation course'] == "completed"][["math score", "reading score", "writing score"]],
          performance_df[performance_df['test preparation course'] == "none"][["math score", "reading score", "writing score"]]))
print('The test preparation course had a significant effect on exam performance.')

Yes, there seems to be a difference.
Ttest_indResult(statistic=array([ 5.70461642,  7.87166354, 10.40917344]), pvalue=array([1.53591346e-08, 9.08178334e-15, 3.68529174e-24]))
The test preparation course had a significant effect on exam performance.


Question 4: Which 2 exam scores are most correlated with each other?

In [ ]:
print(performance_df.corr())
print('Writing and Reading scores are most correlated with each other.')

               math score  reading score  writing score
math score       1.000000       0.817580       0.802642
reading score    0.817580       1.000000       0.954598
writing score    0.802642       0.954598       1.000000
Writing and Reading scores are most correlated with each other.
